Python for Everyone!<br/>[Oregon Curriculum Network](http://4dsolutions.net/ocn/)

# Pattern Verification with Python

There's a real difference between verifying a suspected identity, in this case involving arctangents and phi ($\phi$), and providing an algebraic proof.

My impression is some of the Ramanujan convergent series have been verified, which is easier to do through brute force computation, rather than proved algebraically.

Consider this monster for example:

$$\frac{1}{\pi} = \frac{2\sqrt{2}}{9801} \sum^\infty_{k=0} \frac{(4k)!(1103+26390k)}{(k!)^4 396^{4k}}$$

Has someone provided an algebraic proof?

In the absence of proof, there's still [computational verification](Extended%20Precision.ipynb).

### The Pattern

Here's the pattern David Koski was discovering, using a calculator:

```

    from math import atan as arctan
    Ø = (1 + rt2(5))/2
    arctan( Ø ** 1) -  arctan( Ø ** -1) == arctan(1/2)
    arctan( Ø **-1) -  arctan( Ø ** -3) == arctan(1/3)
    arctan( Ø **-3) -  arctan( Ø ** -5) == arctan(1/7)
    arctan( Ø **-5) -  arctan( Ø ** -7) == arctan(1/18)
    arctan( Ø **-7) -  arctan( Ø ** -9) == arctan(1/47)
    arctan( Ø **-9) -  arctan( Ø **-11) == arctan(1/123)

```
. . .

Given the limitations of floating point numbers, we'll switch to [gmpy2](https://gmpy2.readthedocs.io/en/latest/), which, unlike the Decimal type, supports arctan.  

We may enlist the `unittest.TestCase` method `assertAlmostEqual` to verify the pattern so far.

I'm organizing my tests to run inside of the `verify` function.  That's slightly unusual.  [Here's a link](https://docs.python.org/3/library/unittest.html#organizing-test-code) to the documentation.

In [1]:
import gmpy2
from gmpy2 import atan2 as arctan, atan as at
from unittest import TestCase, TextTestRunner, TestSuite

# arctan(y, x) = arctan(y/x)
# at(x) = atan(x)

gmpy2.get_context().precision=300
Ø = (1 + gmpy2.sqrt(5))/2
    
class TestArcTan(TestCase):
    
    def test_claim(self):
        self.assertAlmostEqual(at(Ø) - arctan(1, Ø), arctan(1,2), places=30)
        self.assertAlmostEqual(arctan( 1, Ø) - arctan(1, Ø** 3), arctan(1,3), places=30)
        self.assertAlmostEqual(arctan( 1, Ø**3) - arctan(1, Ø** 5), arctan(1,7), places=30)
        self.assertAlmostEqual(arctan( 1, Ø**5) - arctan(1, Ø** 7), arctan(1,18), places=30)
        self.assertAlmostEqual(arctan( 1, Ø**7) - arctan(1, Ø** 9), arctan(1,47), places=30)
        self.assertAlmostEqual(arctan( 1, Ø**9) - arctan(1, Ø**11), arctan(1,123), places=30) 
        
def verify():
    """
    how true are these expressions?
    
    using a function to stuff an empty TestSuite, taking from
    an existing class, is not the usual pattern
    """
    def suite():
        suite = TestSuite()
        suite.addTest(TestArcTan('test_claim'))
        return suite
        
    runner = TextTestRunner()
    runner.run(suite())
    
verify()

.
----------------------------------------------------------------------
Ran 1 test in 0.004s

OK


By inspection, the pattern is fairly clear, but for the sequence of fractions 1/2, 1/3, 1/7, 1/18... in the resulting term.  Where is that coming from?  How might we generate this sequence by following some rule?

A useful resource in such cases is the [On-Line Encyclopedia of Integer Sequences](http://oeis.org/), which comes through for us when we search on just those denominators.  We find this is [a "bisection of the Lucas Numbers"](http://oeis.org/A005248).

We learn from this web page that this sequence may be generated as the sum of consecutive odd-position Fibonacci numbers.  

For example:

In [2]:
def Fibo(a=1, b=0):
    # 1, 0, 1, 1, 2, 3, 5, 8, 13, 21...
    while True:
        yield a
        a, b = b, a + b
        
def A005248():
    """
    Skip half the Fibs, sum consecutive terms from 
    the other half:
    
    1 + 1, 1 + 2, 2 + 5, 5 + 13...
    """
    fibo_0 = Fibo()
    fibo_1 = Fibo()

    # advance 2 positions
    next(fibo_1); next(fibo_1)  #1, 0
    
    while True:
        yield next(fibo_0) + next(fibo_1) # 1 + 1, 1 + 2, 2 + 5...
        next(fibo_0) # skip over next Fib
        next(fibo_1) # skip over next Fib

a005248 = A005248()
print([next(a005248) for _ in range(20)])

[2, 3, 7, 18, 47, 123, 322, 843, 2207, 5778, 15127, 39603, 103682, 271443, 710647, 1860498, 4870847, 12752043, 33385282, 87403803]


Now we have the ability to build successive arctan expressions and verify them.  Rather then depend on unittest, lets simply compute the floating point difference between the two sides of the equation.  If it stays very small, we will consider the expression verified.

In [3]:
def build_expr():
    a005248 = A005248()
    exp1 = 1
    exp2 = -1
    template = "(at(Ø**{}) - at(Ø**{})) - arctan(1, {})"
    while True:
        yield template.format(exp1, exp2, next(a005248))
        exp1 -= 2
        exp2 -= 2

expr_gen = build_expr()

for _ in range(20):
    expr = next(expr_gen)
    print("Expr: {}".format(expr))
    print("Diff: {0:4.20e}".format(eval(expr)))

Expr: (at(Ø**1) - at(Ø**-1)) - arctan(1, 2)
Diff: 2.45454673264886327655e-91
Expr: (at(Ø**-1) - at(Ø**-3)) - arctan(1, 3)
Diff: -2.45454673264886327655e-91
Expr: (at(Ø**-3) - at(Ø**-5)) - arctan(1, 7)
Diff: 0.00000000000000000000e+00
Expr: (at(Ø**-5) - at(Ø**-7)) - arctan(1, 18)
Diff: -3.06818341581107909568e-92
Expr: (at(Ø**-7) - at(Ø**-9)) - arctan(1, 47)
Diff: -6.13636683162215819137e-92
Expr: (at(Ø**-9) - at(Ø**-11)) - arctan(1, 123)
Diff: -7.67045853952769773921e-93
Expr: (at(Ø**-11) - at(Ø**-13)) - arctan(1, 322)
Diff: -3.83522926976384886961e-93
Expr: (at(Ø**-13) - at(Ø**-15)) - arctan(1, 843)
Diff: -3.83522926976384886961e-93
Expr: (at(Ø**-15) - at(Ø**-17)) - arctan(1, 2207)
Diff: -2.15731646424216498915e-93
Expr: (at(Ø**-17) - at(Ø**-19)) - arctan(1, 5778)
Diff: -4.79403658720481108701e-94
Expr: (at(Ø**-19) - at(Ø**-21)) - arctan(1, 15127)
Diff: -2.99627286700300692938e-94
Expr: (at(Ø**-21) - at(Ø**-23)) - arctan(1, 39603)
Diff: -1.19850914680120277175e-94
Expr: (at(Ø**-23) - 

Looking good!

*For Further Reading*:

* [Note on edu-sig](https://mail.python.org/pipermail/edu-sig/2018-October/012024.html) (October, 2018)
* [Pi Day Fun](Pi%20Day%20Fun.ipynb) (Jupyter Notebook, March, 2016) 
* [Koski Paper @ Coffee Shops Network](http://coffeeshopsnet.blogspot.com/2017/06/koski-paper.html) (2015)